In [20]:
import pandas as pd
import numpy as np

'''
Loading Gensim and nltk libraries
'''
# pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

import nltk
nltk.download('wordnet')

#For printing the values without calling print() function everytime
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading package wordnet to C:\Users\SrinivasRengarajan
[nltk_data]     .DESKTOP-C5Q3RJS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
# !pip install pyLDAvis

In [45]:
# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
reviews_df=pd.read_csv("Reviews_of_Amazon_Products.csv")

In [4]:
reviews_df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/revie...,Didnt know how much i'd use a kindle so went f...,Great for the price,johnnyjojojo,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/...,I am 100 happy with my purchase. I caught it o...,A Great Buy,Kdperry,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...


In [18]:
#Splitting the data into training and testing in 80/20 proportions
reviews_train_df=reviews_df.sample(frac=0.8,random_state=599) #random state is used for setting the seed value for reproducibility
reviews_test_df=reviews_df.drop(reviews_train_df.index)

#Resetting the index for clear understanding
reviews_train_df=reviews_train_df.reset_index(drop=True)
reviews_test_df=reviews_test_df.reset_index(drop=True)

In [21]:
len(reviews_train_df)
len(reviews_test_df)

4000

1000

In [5]:
'''
A function to perform the pre processing steps on the entire dataset
'''
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [22]:
stemmer = SnowballStemmer("english")

processed_docs = []

for doc in reviews_train_df['reviews.text']:
    processed_docs.append(preprocess(doc))

In [23]:
print(processed_docs[5])

['daughter', 'love', 'tablet', 'bigger', 'screen', 'featur', 'enjoy', 'kindl', 'price', 'right', 'easi', 'abl', 'app', 'transfer', 'tablet', 'time', 'great', 'invest']


In [24]:
'''
Creating a dictionary from 'processed_docs' containing the number of times a word appears 
in the training set using gensim.corpora.Dictionary and calling it as 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

"\nCreating a dictionary from 'processed_docs' containing the number of times a word appears \nin the training set using gensim.corpora.Dictionary and calling it as 'dictionary'\n"

In [25]:
'''
Checking the dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

'\nChecking the dictionary created\n'

0 age
1 connect
2 improv
3 ipad
4 larger
5 notic
6 prefer
7 replac
8 screen
9 wifi
10 beginn


In [26]:
'''
This is an OPTIONAL STEP
Removing very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n= 100000)

'\nThis is an OPTIONAL STEP\nRemoving very rare and very common words:\n\n- words appearing less than 15 times\n- words appearing in more than 10% of all documents\n'

In [27]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

"\nCreate the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many\nwords and how many times those words appear. Save this to 'bow_corpus'\n"

In [28]:
'''
Previewing BOW for our sample preprocessed document
'''
document_num = 20
bow_doc_x = bow_corpus[document_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

'\nPreviewing BOW for our sample preprocessed document\n'

Word 42 ("enjoy") appears 1 time.
Word 58 ("light") appears 1 time.
Word 130 ("case") appears 1 time.
Word 131 ("choos") appears 1 time.
Word 132 ("christma") appears 1 time.
Word 133 ("level") appears 1 time.
Word 134 ("lock") appears 1 time.
Word 135 ("parent") appears 1 time.
Word 136 ("timer") appears 1 time.
Word 137 ("year") appears 2 time.


In [29]:
'''
Training our LDA model using gensim.models.LdaMulticore
'''
# TODO
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 7, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2, random_state=599)

'\nTraining our LDA model using gensim.models.LdaMulticore\n'

In [30]:
'''
For each topic, we will explore the words occuring in that topic and its relative weight
'''
for idx, words in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, words ))
    print("\n")

'\nFor each topic, we will explore the words occuring in that topic and its relative weight\n'

Topic: 0 
Words: 0.040*"sound" + 0.032*"time" + 0.030*"watch" + 0.029*"video" + 0.025*"qualiti" + 0.022*"plus" + 0.022*"speaker" + 0.021*"learn" + 0.017*"thing" + 0.017*"better"


Topic: 1 
Words: 0.073*"kid" + 0.055*"price" + 0.022*"game" + 0.021*"control" + 0.020*"awesom" + 0.019*"happi" + 0.018*"parent" + 0.016*"play" + 0.015*"movi" + 0.014*"prime"


Topic: 2 
Words: 0.059*"alexa" + 0.039*"music" + 0.026*"home" + 0.019*"play" + 0.019*"devic" + 0.019*"smart" + 0.017*"light" + 0.017*"video" + 0.015*"famili" + 0.015*"sound"


Topic: 3 
Words: 0.048*"book" + 0.029*"best" + 0.028*"recommend" + 0.022*"reader" + 0.017*"light" + 0.017*"need" + 0.016*"upgrad" + 0.015*"want" + 0.014*"littl" + 0.014*"devic"


Topic: 4 
Words: 0.049*"year" + 0.024*"charg" + 0.021*"need" + 0.020*"ipad" + 0.018*"game" + 0.017*"replac" + 0.017*"book" + 0.016*"daughter" + 0.016*"gift" + 0.016*"christma"


Topic: 5 
Words: 0.063*"perfect" + 0.029*"item" + 0.027*"size" + 0.024*"gift" + 0.021*"recommend" + 0.020*"ligh

In [46]:
# Visualizing the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.203792 -0.057530       1        1  21.410149
6     -0.011270 -0.054711       2        1  16.763704
4      0.094890  0.062569       3        1  15.716298
3      0.107329 -0.108888       4        1  12.915085
1      0.057792  0.144812       5        1  12.278526
0     -0.129534  0.075299       6        1  11.325906
5      0.084584 -0.061550       7        1   9.590333, topic_info=        Term        Freq       Total Category  logprob  loglift
112      kid  341.000000  341.000000  Default  30.0000  30.0000
26     alexa  429.000000  429.000000  Default  29.0000  29.0000
211  perfect  231.000000  231.000000  Default  28.0000  28.0000
44     price  404.000000  404.000000  Default  27.0000  27.0000
63     sound  276.000000  276.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
36     thing   31.987028  247.761612   Topic7  -4.5170   0.2973
8     screen   35.031635  434.001678   Topic7  -4.4261  -0.1724
27      amaz   23.047392  112.380058   Topic7  -4.8448   0.7601
160     need   27.581743  299.527100   Topic7  -4.6652  -0.0406
42     enjoy   21.734156  222.349609   Topic7  -4.9034   0.0190

[386 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
38        1  0.452756     abl
38        2  0.047040     abl
38        3  0.088199     abl
38        4  0.264598     abl
38        5  0.088199     abl
...     ...       ...     ...
147       2  0.149013  youtub
147       4  0.042575  youtub
147       5  0.021288  youtub
147       6  0.489615  youtub
147       7  0.085150  youtub

[1323 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 5, 4, 2, 1, 6])

In [32]:
reviews_test_df.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-05-27T00:00:00Z,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,Solid entry level Kindle. Great for kids. Gift...,Solid entry-level Kindle. Great for kids,Johnnyblack,https://www.newegg.com/Product/Product.aspx%25...
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/revie...,"Love my kindle, makes reading at night so easy...",feels like a real book,Sean242,https://www.newegg.com/Product/Product.aspx%25...
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2018-10-09T00:00:00Z,True,135864424.0,4,4,https://redsky.target.com/groot-domain-api/v1/...,This is a nice bare-bones reading device. Easy...,"Easy to use, bare-bones reading device",Nibbler,https://www.newegg.com/Product/Product.aspx%25...
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,2017-10-10T00:00:00Z,True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,The sales people are really nice and knowledge...,Great product,richard,https://www.newegg.com/Product/Product.aspx%25...


In [39]:
sample_test_document = reviews_test_df['reviews.text'].iloc[50]

In [40]:
print("Sample Test Document:\n", sample_test_document)

Sample Test Document:
 I use it in my office as a personal assistant. Not just for the home


In [41]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(sample_test_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.8283374309539795	 Topic: 0.059*"alexa" + 0.039*"music" + 0.026*"home" + 0.019*"play" + 0.019*"devic"
Score: 0.028660831972956657	 Topic: 0.040*"sound" + 0.032*"time" + 0.030*"watch" + 0.029*"video" + 0.025*"qualiti"
Score: 0.02861718088388443	 Topic: 0.047*"screen" + 0.033*"devic" + 0.018*"better" + 0.017*"app" + 0.017*"play"
Score: 0.02861364558339119	 Topic: 0.049*"year" + 0.024*"charg" + 0.021*"need" + 0.020*"ipad" + 0.018*"game"
Score: 0.028600841760635376	 Topic: 0.073*"kid" + 0.055*"price" + 0.022*"game" + 0.021*"control" + 0.020*"awesom"
Score: 0.0285891592502594	 Topic: 0.048*"book" + 0.029*"best" + 0.028*"recommend" + 0.022*"reader" + 0.017*"light"
Score: 0.028580958023667336	 Topic: 0.063*"perfect" + 0.029*"item" + 0.027*"size" + 0.024*"gift" + 0.021*"recommend"
